In [1]:
#import pandas library

import pandas as pd

In [2]:
#load dataset

dataset=pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
# Categorical input data

dataset=pd.get_dummies(dataset,drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [4]:
# Drop UserID column 
# axis =1 for column
# axis =0 for row

dataset=dataset.drop("User ID",axis=1)
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [5]:
# Balanced/Unbalanced data

dataset['Purchased'].value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [6]:
# View dataset column names

dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [7]:
# Split Input data

independent=dataset[['Age', 'EstimatedSalary', 'Gender_Male']]
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [8]:
# Split Output data

dependent=dataset[[ 'Purchased']]
dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [9]:
# split training and testing data

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent, dependent, test_size=1/3,random_state=0)


In [10]:
# Standardization

from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [37]:
# GridSearchCV Model Creation - SVM
# classifier=SVC(kernel=;'rbf', random_state=0)

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
param_grid={'C': [0.1, 1, 10, 100, 1000],
            'kernel': ['rbf','linear']
           }
grid=GridSearchCV(SVC(probability=True),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
# SVC(probability=True) - resolved attribute error in roc_auc_score 
grid.fit(x_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


F:\Anaconda\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'kernel': ['rbf', 'linear']},
             scoring='f1_weighted', verbose=3)

In [31]:
# Hyper Tuning parameter

result=grid.cv_results_

# Evaluation metrics

y_pred=grid.predict(x_test)

In [32]:
# Confusion Matrix

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
cm

array([[77,  8],
       [ 4, 45]], dtype=int64)

In [33]:
# Classification report for test data

from sklearn.metrics import classification_report
cr=classification_report(y_test,y_pred)
print(cr)

              precision    recall  f1-score   support

           0       0.95      0.91      0.93        85
           1       0.85      0.92      0.88        49

    accuracy                           0.91       134
   macro avg       0.90      0.91      0.91       134
weighted avg       0.91      0.91      0.91       134



In [34]:
# View in dataFrame

table=pd.DataFrame.from_dict(result)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.040624,0.012499,0.015625,6.810597e-07,0.1,rbf,"{'C': 0.1, 'kernel': 'rbf'}",0.843811,0.849057,0.791152,0.981233,0.961755,0.885402,0.073413,4
1,0.018747,0.006250,0.018752,6.250883e-03,0.1,linear,"{'C': 0.1, 'kernel': 'linear'}",0.751984,0.770070,0.698235,0.923510,0.859176,0.800595,0.080389,10
2,0.034934,0.003125,0.018043,3.581040e-03,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.887346,0.886792,0.850543,0.944161,0.962264,0.906221,0.041022,1
3,0.021300,0.003077,0.019989,3.946270e-03,1,linear,"{'C': 1, 'kernel': 'linear'}",0.776290,0.770070,0.698235,0.923510,0.901744,0.813970,0.085375,9
4,0.023383,0.006146,0.020788,4.396521e-03,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.867478,0.886792,0.869709,0.944161,0.943041,0.902236,0.034431,3
5,0.025183,0.003308,0.017579,4.487970e-03,10,linear,"{'C': 10, 'kernel': 'linear'}",0.776290,0.790949,0.698235,0.923510,0.901744,0.818146,0.083619,6
6,0.029552,0.008073,0.018989,7.057657e-03,100,rbf,"{'C': 100, 'kernel': 'rbf'}",0.867478,0.886792,0.870362,0.944161,0.943041,0.902367,0.034308,2
7,0.075753,0.006520,0.016389,1.020263e-03,100,linear,"{'C': 100, 'kernel': 'linear'}",0.776290,0.790949,0.698235,0.923510,0.901744,0.818146,0.083619,6
8,0.071335,0.011918,0.016798,2.170700e-03,1000,rbf,"{'C': 1000, 'kernel': 'rbf'}",0.847141,0.866968,0.851527,0.925272,0.923510,0.882884,0.034529,5
9,0.441449,0.063324,0.010975,6.232588e-03,1000,linear,"{'C': 1000, 'kernel': 'linear'}",0.776290,0.790949,0.698235,0.923510,0.901744,0.818146,0.083619,6


In [35]:
# f1 score
# check with classification report 

from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,y_pred,average='weighted')
print("The f1_macro value for best parameter {}".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'C': 1, 'kernel': 'rbf'} 0.9111247448089108


In [36]:
# roc_auc_score

from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.9711884753901561